# Lab1: Finetuning HuggingFace models with Amazon SageMaker
### Binary Classification with `Trainer` and `imdb` dataset

# Introduction

Welcome to the first Lab and our end-to-end multi-class Text-Classification example. In this Lab, we will use the Hugging Faces `transformers` and `datasets` library together with Amazon SageMaker to fine-tune a pre-trained transformer on text classification. In particular, the pre-trained model will be fine-tuned using the `amazon_us_reviews` dataset. To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on. 

# Development Environment and Permissions 

## Installation

_*Note:* we install the required libraries from Hugging Face and AWS. Additionally, we make sure we have a compatible PyTorch version installed_

In [1]:
!pip install "sagemaker>=2.48.0" --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached sagemaker-2.56.0-py2.py3-none-any.whl
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.49.1
    Uninstalling sagemaker-2.49.1:
      Successfully uninstalled sagemaker-2.49.1
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


## Permissions

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [2]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::558105141721:role/sagemaker_execution_role
sagemaker bucket: sagemaker-us-east-1-558105141721
sagemaker session region: us-east-1


# Preparing the dataset

As dataset are we going to use the [amazon_us_reviews](https://huggingface.co/datasets/amazon_us_reviews).

the dataset contains the customer review text with accompanying metadata, consisting of three major components:

1. A collection of reviews written in the Amazon.com marketplace and associated metadata from 1995 until 2015. This is intended to facilitate study into the properties (and the evolution) of customer reviews potentially including how people evaluate and express their experiences with respect to products at scale. (130M+ customer reviews)
2. A collection of reviews about products in multiple languages from different Amazon marketplaces, intended to facilitate analysis of customers’ perception of the same products and wider consumer preferences across languages and countries. (200K+ customer reviews in 5 countries)
3. A collection of reviews that have been identified as non-compliant with respect to Amazon policies. This is intended to provide a reference dataset for research on detecting promotional or biased reviews. (several thousand customer reviews). This part of the dataset is distributed separately and is available upon request – please contact the email address below if you are interested in obtaining this dataset.  

_https://s3.amazonaws.com/amazon-reviews-pds/readme.html_

The Lab already includes a downsampled train dataset `data/amazon_us_reviews_apparel_v1_00_train.json` and downsampled test dataset `data/amazon_us_reviews_apparel_v1_00_test.json`. The train dataset contains 29750 rows and the test dataset 5250. 
For the dataset files the `Apparel_v1_00` split was used. 

In Addition to the dataset files the repository also contains the script of how these datasets has been generated. You can find the script at `data/create_dataset.py`. You can use this to change for example the size or category split of the reviews.

In [3]:
!pygmentize ../data/create_dataset.py

import datasets
from datasets import load_dataset


assert float(datasets.__version__[:3]) >= 1.8, "`datasets 1.8.0` or higher need to be installed to generate dataset"


"""
This script is creating a sample dataset for the workshop using the `datasets` library and the "amazon_us_reviews" 
dataset. You can configure which dataset split should be used, by default it is the apparel split. You can also
configure the size of the dataset, which is generated. The script creates 2 json files one for training and one for
testing, which need to be uploaded to s3 for the workshop.
"""


# configuration
dataset_name = "amazon_us_reviews"
dataset_split = "Apparel_v1_00"
train_dataset_length = 35_000
test_split_size = 0.15


# load dataset using datasets library, using the Apperal split.
# full information can be found here: https://huggingface.co/datasets/amazon_us_reviews
dataset = load_dataset("amazon_us_reviews", "Apparel_v1_00")

# since there is only a "tran" split assign it as dataset
datase

## Uploading data to `sagemaker_session_bucket`

Upload the `dataset` files to the default bucket in Amazon S3

In [8]:
import os
from sagemaker.s3 import S3Uploader

local_train_dataset = "amazon_us_reviews_apparel_v1_00_train.json"
local_test_dataset = "amazon_us_reviews_apparel_v1_00_test.json"

# s3 uris for datasets
remote_train_dataset = f"s3://{sess.default_bucket()}/lab1/data"
remote_test_dataset = f"s3://{sess.default_bucket()}/lab1/data"

# upload datasets
S3Uploader.upload(os.path.join("../data",local_train_dataset),remote_train_dataset)
S3Uploader.upload(os.path.join("../data",local_test_dataset),remote_test_dataset)

print(f"train dataset uploaded to: {remote_train_dataset}/{local_train_dataset}")
print(f"test dataset uploaded to: {remote_test_dataset}/{local_test_dataset}")


train dataset uploaded to: s3://sagemaker-us-east-1-558105141721/lab1/data/amazon_us_reviews_apparel_v1_00_train.json
test dataset uploaded to: s3://sagemaker-us-east-1-558105141721/lab1/data/amazon_us_reviews_apparel_v1_00_test.json


# Fine-tuning & starting Sagemaker Training Job

In order to create our sagemaker training job we need an `HuggingFace` Estimator. The Estimator handles all end-to-end Amazon SageMaker training and deployment tasks. In the Estimator we define, which fine-tuning script (`entry_point`) should be used, which `instance_type` should be used, which `hyperparameters` are passed in .....



```python
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            transformers_version='4.4',
                            pytorch_version='1.6',
                            py_version='py36',
                            role=role,
                            hyperparameters = {'epochs': 1,
                                               'train_batch_size': 32,
                                               'model_name':'distilbert-base-uncased'
                                                })
```

When we create a SageMaker training job, SageMaker takes care of starting and managing the required ec2 instances for us, providing the fine-tuning script `train.py` and downloading the data from our `sagemaker_session_bucket` into the container at `/opt/ml/input/data`. When starting the training SageMaer executes the following command:

```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

The CLI arguments you see are passed in as `hyperparameters`, when creating the `HuggingFace` estimator.

Sagemaker is also providing useful properties about the training environment through various environment variables, including the following:

* `SM_MODEL_DIR`: A string that represents the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting.

* `SM_NUM_GPUS`: An integer representing the number of GPUs available to the host.

* `SM_CHANNEL_XXXX:` A string that represents the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the HuggingFace estimator’s fit call, named `train` and `test`, the environment variables `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` are set.


To run your training job locally you can define `instance_type='local'` or `instance_type='local_gpu'` for gpu usage. _Note: this does not working within SageMaker Studio_


## Creating an Estimator and start a training job

The training script that performs fine tuning is located here: `training/scripts/train.py`. Navigate to the source code location and open the `train.py` file. You can also go through it's contents by executing the cell below.

In [9]:
!pygmentize ../scripts/train.py

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset
import random
import logging
import sys
import argparse
import os
import torch

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=500)
    parser.add_argument("--model_id", type=str)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    parser.add_argument("--train_file", type=str, default="amazon_us_reviews_apparel_v1_00_train.json")
    parser.add_argument("--test_file", type=str, default="amazon_us_re

In [10]:
from sagemaker.huggingface import HuggingFace
import time

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 3,                          # number of training epochs
                 'train_batch_size': 16,               # batch size for training
                 'eval_batch_size': 32,                # batch size for evaluation
                 'learning_rate': 3e-5,                # learning rate used during training
                 'model_id':'distilbert-base-uncased', # pre-trained model
                 'train_file': local_train_dataset,    # training dataset
                 'test_file': local_test_dataset,      # test dataset
                 }

List of supported models: https://huggingface.co/models?library=pytorch,transformers&sort=downloads

We create a `metric_definition` dictionary that contains regex-based definitions that will be used to parse the job logs and extract metrics. You can read more about parsing the cloudwatch logs [here](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html).

In [11]:
metric_definitions=[
    {'Name': 'eval_loss',               'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_accuracy',           'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_f1',                 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_precision',          'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"}]

In [14]:
# define Training Job Name 
job_name = f'huggingface-workshop-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # fine-tuning script used in training jon
    source_dir           = '../scripts',      # directory where fine-tuning script is stored
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.6',             # the transformers version used in the training job
    pytorch_version      = '1.7',             # the pytorch_version version used in the training job
    py_version           = 'py36',            # the python version used in the training job
    hyperparameters      = hyperparameters,   # the hyperparameter used for running the training job
    metric_definitions   = metric_definitions # the metrics regex definitions to extract logs
)

In [15]:
# define a data input dictonary with our uploaded s3 uris
training_data = {
    'train': remote_train_dataset,
    'test': remote_test_dataset
}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(training_data, wait=True)

2021-08-30 13:26:58 Starting - Starting the training job...
2021-08-30 13:27:21 Starting - Launching requested ML instancesProfilerReport-1630330018: InProgress
...
2021-08-30 13:27:55 Starting - Preparing the instances for training............
2021-08-30 13:29:58 Downloading - Downloading input data
2021-08-30 13:29:58 Training - Downloading the training image..................
2021-08-30 13:32:51 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-30 13:32:51,834 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-30 13:32:51,858 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-30 13:32:51,865 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-30 13:32:52,253 sagemaker-training-toolkit INFO     Invoking user script

Training

UnexpectedStatusException: Error for Training job huggingface-workshop-2021-08-30-13-26-5-2021-08-30-13-26-57-988: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train.py --epochs 3 --eval_batch_size 32 --learning_rate 3e-05 --model_id distilbert-base-uncased --test_file amazon_us_reviews_apparel_v1_00_test.json --train_batch_size 16 --train_file amazon_us_reviews_apparel_v1_00_train.json"
0 tables [00:00, ? tables/s]                            0 tables [00:00, ? tables/s]                            Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]Downloading: 100%|ââââââââââ| 483/483 [00:00<00:00, 649kB/s]
Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]Downloading: 100%|ââââââââââ| 232k/232k [00:00<00:00, 37.2MB/s]
Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]Downloading: 100%|ââââââââââ| 466k/466k [00:00<00:00, 40.4MB/s]
Downloading:   0%|        

# Accessing Logs

In [ ]:
# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

# Accessing Training Metrics

In [17]:
from sagemaker import TrainingJobAnalytics

# Captured metrics can be accessed as a Pandas dataframe
training_job_name = huggingface_estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df

Training jobname: huggingface-workshop-2021-08-26-17-53-4-2021-08-26-17-53-43-765


,timestamp,metric_name,value
0,0.0,eval_loss,0.206314
1,540.0,eval_loss,0.235827
2,1140.0,eval_loss,0.280385
3,0.0,eval_accuracy,0.918100
4,540.0,eval_accuracy,0.932200
5,1140.0,eval_accuracy,0.934700
6,0.0,eval_f1,0.921273
7,540.0,eval_f1,0.932751
8,1140.0,eval_f1,0.935031
9,0.0,eval_precision,0.890375


## Bonus: Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

Then, we use the returned predictor object to call the endpoint.

In [ ]:
sentiment_input= {"inputs":"I love using the new Inference DLC."}

predictor.predict(sentiment_input)

Finally, we delete the endpoint again.

In [12]:
predictor.delete_endpoint()